In [1]:
from kgtk.functions import kgtk, kypher
import os
import gzip
from kgtk.io.kgtkreader import KgtkReader
import json
import sys
from pathlib import Path
import pandas as pd

In [2]:
os.environ['STORE'] = '/Volumes/saggu-ssd/wikidata.sqlite3.db'
os.environ['sitelinks'] = '/Volumes/saggu-ssd/sitelinks.tsv.gz'

In [3]:
!kgtk --debug query --gc $STORE --show-cache

Graph Cache:
DB file: /Volumes/saggu-ssd/wikidata.sqlite3.db
  size:  284.35 GB   	free:  0 Bytes   	modified:  2022-08-01 10:30:31

KGTK File Information:
alias:
  size:  179.00 MB   	modified:  2022-06-11 12:47:43   	graph:  graph_3
claims:
  size:  15.25 GB   	modified:  2022-06-11 15:03:49   	graph:  graph_5
classvizedge:
  size:  595.03 MB   	modified:  2022-05-20 17:24:07   	graph:  graph_8
classviznode:
  size:  502.27 MB   	modified:  2022-06-11 20:08:05   	graph:  graph_9
datatypes:
  size:  54.74 KB   	modified:  2022-06-11 12:49:29   	graph:  graph_6
description:
  size:  417.24 MB   	modified:  2022-06-11 12:49:25   	graph:  graph_4
l_d_pgr_ud:
  size:  1.33 GB   	modified:  2022-06-11 21:22:36   	graph:  graph_10
label:
  size:  662.80 MB   	modified:  2022-06-11 12:47:00   	graph:  graph_1
pagerank_undirected:
  size:  1.52 GB   	modified:  2022-06-09 14:01:23   	graph:  graph_2
qualifiers:
  size:  4.09 GB   	modified:  2022-06-11 13:18:44   	graph:  graph_7
sitelinks:
 

In [ ]:
kypher(""" --debug
            -i claims -i qualifiers -i label --gc $STORE
            --match 'claims: (n1:Q12073038)-[l:P26]->(n2),
                     label: (n1:Q12073038)-[]->(n1_label),
                     label: (n2)-[]->(n2_label)'
            --opt 'qualifiers: (l)-[:P580]->(start_time)'
            --return 'n1 as node1, 
                      kgtk_lqstring_text(n1_label) as node1_label, 
                      "married to" as label, 
                      n2 as node2, 
                      kgtk_lqstring_text(n2_label) as node2_label, 
                      substr(kgtk_date_date(start_time), 2) as start_time'
            --order-by 'start_time desc'
            
""")

In [ ]:
kypher(""" --debug
            -i claims -i qualifiers --gc $STORE
            --match 'claims: (n1:Q12073038)-[l:P26]->(n2)'
            --opt 'qualifiers: (l)-[:P580]->(start_time)'
            
""")

In [ ]:
kypher(""" --debug
            -i claims --gc $STORE
            --match 'claims: (n1:Q12073038)-[l:P26]->(n2)'
            
""")

In [ ]:
kypher(""" --debug
            -i claims --gc $STORE
            --match 'claims: (n1:Q12073038)-[]->(n2)'
            
""")

In [ ]:
kypher(""" --debug
            -i claims -i qualifiers -i label --gc $STORE
            --match 'claims: (n1)-[l:P26]->(n2),
                     label: (n1)-[]->(n1_label),
                     label: (n2)-[]->(n2_label)'
            --opt 'qualifiers: (l)-[:P580]->(start_time)'
            --return 'n1 as node1, 
                      kgtk_lqstring_text(n1_label) as node1_label, 
                      "married to" as label, 
                      n2 as node2, 
                      kgtk_lqstring_text(n2_label) as node2_label, 
                      substr(kgtk_date_date(start_time), 2) as start_time'
            --order-by 'start_time desc'
            -o /tmp/marriages_in_wikidata.tsv.gz
""")

In [ ]:
!gzcat /tmp/marriages_in_wikidata.tsv.gz | head

In [ ]:
!gzcat /tmp/marriages_in_wikidata.tsv.gz | wc -l

In [ ]:
def read_sitelinks(sitelinks_file):
    f = gzip.open(sitelinks_file, 'rt')
    sitelinks_d = dict()
    for line in f:
        vals = line.split('\t')
        node = vals[1]
        lang = vals[4]
        url = vals[3]
        prop = vals[2]
        if prop == 'wikipedia_sitelink' and lang == 'en':
            sitelinks_d[node] = url
    return sitelinks_d

In [ ]:
sitelinks_d = read_sitelinks(os.environ['sitelinks'])

In [ ]:
sitelinks_d['Q12073038']

In [ ]:
def first_name(name):
    if len(name.split(' ')) == 1:
           return name
    return ' '.join(name.split(' ')[:-1])

In [ ]:
first_name('Karan Singh Grover')

In [ ]:
first_name('Karan')

In [ ]:
def create_marriage_questions(married_file, o_answers, o_no_answers):
    kr = KgtkReader.open(Path(married_file),
                          error_file=sys.stderr,
                          who="married"
                          )
    node1_lbl_idx = kr.column_name_map['node1_label']
    node2_lbl_idx = kr.column_name_map['node2_label']
    node1_idx = kr.column_name_map['node1']
    node2_idx = kr.column_name_map['node2']
    st_idx = kr.column_name_map['start_time']
    seen_questions = set()
    questions_with_answers = []
    questions_no_answers = []
    for row in kr:
        node1 = row[node1_idx]
        node2 = row[node2_idx]
        node1_l = row[node1_lbl_idx]
        node2_l = row[node2_lbl_idx]
        answer = row[st_idx].strip()
        if node1 in sitelinks_d and node2 in sitelinks_d:
            question_1 = f"when did {node1_l} marry {first_name(node2_l)}?"
            question_2 = f"when did {node2_l} marry {first_name(node1_l)}?"
            if question_1 not in seen_questions and question_2 not in seen_questions:
                if answer != '':
                    questions_with_answers.append({'q': question_1, 
                                      'a': answer, 
                                      'n1': node1, 
                                      'n2': node2, 
                                      'n1_label': node1_l, 
                                      'n2_label': node2_l,
                                      'url': sitelinks_d[node1]})
                    questions_with_answers.append({'q': question_2, 
                                      'a': answer, 
                                      'n1': node2, 
                                      'n2': node1, 
                                      'n1_label': node2_l, 
                                      'n2_label': node1_l,
                                      'url': sitelinks_d[node2]})
                else:
                    questions_no_answers.append({'q': question_1, 
                                      'a': answer, 
                                      'n1': node1, 
                                      'n2': node2, 
                                      'n1_label': node1_l, 
                                      'n2_label': node2_l,
                                      'url': sitelinks_d[node1]})
                    questions_no_answers.append({'q': question_2, 
                                      'a': answer, 
                                      'n1': node2, 
                                      'n2': node1, 
                                      'n1_label': node2_l, 
                                      'n2_label': node1_l,
                                      'url': sitelinks_d[node2]})

                seen_questions.add(question_1)
                seen_questions.add(question_2)
        elif node1 in sitelinks_d:
            question = f"when did {node1_l} marry {first_name(node2_l)}?"     
            if question not in seen_questions:
                if answer != '':
                    questions_with_answers.append({'q': question, 
                                      'a': answer, 
                                      'n1': node1, 
                                      'n2': node2, 
                                      'n1_label': node1_l, 
                                      'n2_label': node2_l,
                                      'url': sitelinks_d[node1]})

                else:
                    questions_no_answers.append({'q': question, 
                                      'a': answer, 
                                      'n1': node1, 
                                      'n2': node2, 
                                      'n1_label': node1_l, 
                                      'n2_label': node2_l,
                                      'url': sitelinks_d[node1]})
            seen_questions.add(question)
        elif node2 in sitelinks_d:
            question = f"when did {node2_l} marry {first_name(node1_l)}?"     
            if question not in seen_questions:
                if answer != '':
                    questions_with_answers.append({'q': question, 
                                      'a': answer, 
                                      'n1': node2, 
                                      'n2': node1, 
                                      'n1_label': node2_l, 
                                      'n2_label': node1_l,
                                      'url': sitelinks_d[node2]})

                else:
                    questions_no_answers.append({'q': question, 
                                      'a': answer, 
                                      'n1': node2, 
                                      'n2': node1, 
                                      'n1_label': node2_l, 
                                      'n2_label': node1_l,
                                      'url': sitelinks_d[node2]})
            seen_questions.add(question)
    open(o_answers, 'w').write(json.dumps(questions_with_answers))
    open(o_no_answers, 'w').write(json.dumps(questions_no_answers))
    print(len(questions_with_answers))
    print(len(questions_no_answers))
    kr.close()

In [ ]:
create_marriage_questions('/tmp/marriages_in_wikidata.tsv.gz',
                          '/tmp/marriages_in_wikidata.json',
                         '/tmp/marriages_in_wikidata_no_answers.json')

In [ ]:
mq = json.load(open('/tmp/marriages_in_wikidata.json'))

In [ ]:
for h in mq:
    if h['n1'] == 'Q12073038' or h['n2'] == 'Q12073038':
        print(h)

## Divorces

In [ ]:
kypher(""" --debug
            -i claims -i qualifiers -i label -i sitelinks --gc $STORE
            --match 'claims: (n1)-[l:P26]->(n2),
                     label: (n1)-[]->(n1_label),
                     label: (n2)-[]->(n2_label),
                     sitelinks: (n1)-[w:wikipedia_sitelink]->(wikipedia_url)'
            --opt 'qualifiers: (l)-[:P582]->(endd_time)'
            --where 'substr(kgtk_date_date(endd_time), 2) < "2019-12-31" AND w.lang = "en"'
            --return 'n1 as node1, 
                      kgtk_lqstring_text(n1_label) as node1_label, 
                      "married to" as label, 
                      n2 as node2, 
                      kgtk_lqstring_text(n2_label) as node2_label, 
                      substr(kgtk_date_date(endd_time), 2) as end_time'
            --order-by 'end_time desc'
            -o '../data/divorces_in_wikidata.tsv.gz'
""")

In [ ]:
divorces_df  = pd.read_csv('../data/divorces_in_wikidata.tsv.gz', sep='\t').fillna('')

In [ ]:
print(len(divorces_df), len(divorces_df[divorces_df['end_time'] == '']), len(divorces_df[divorces_df['end_time'] != '']))

## Position held (P39)

In [ ]:
kypher(""" --debug
            -i claims -i qualifiers -i label -i $sitelinks --as sitelinks --gc $STORE
            --match 'claims: (n1)-[l:P39]->(n2),
                     label: (n1)-[]->(n1_label),
                     label: (n2)-[]->(n2_label),
                     sitelinks: (n1)-[w:wikipedia_sitelink]->(wikipedia_url)'
            --opt 'qualifiers: (l)-[:P580]->(st_time)'
            --where 'substr(kgtk_date_date(st_time), 2) < "2019-12-31" AND w.lang = "en"'
            --return 'n1 as node1, 
                      kgtk_lqstring_text(n1_label) as node1_label, 
                      "position held" as label, 
                      n2 as node2, 
                      kgtk_lqstring_text(n2_label) as node2_label, 
                      substr(kgtk_date_date(st_time), 2) as start_time,
                      wikipedia_url as wiki_url'
            --order-by 'start_time desc'
            -o '../data/position_held_wikidata.tsv.gz'

""")

In [ ]:
position_df  = pd.read_csv('../data/position_held_wikidata.tsv.gz', sep='\t').fillna('')

In [ ]:
print(len(position_df), len(position_df[position_df['start_time'] == '']), len(position_df[position_df['start_time'] != '']))

## Member of sports team (P54)

In [4]:
kypher(""" --debug
            -i claims -i qualifiers -i label -i sitelinks --gc $STORE
            --match 'claims: (n1)-[l:P54]->(n2),
                     label: (n1)-[]->(n1_label),
                     label: (n2)-[]->(n2_label),
                     sitelinks: (n1)-[w:wikipedia_sitelink]->(wikipedia_url)'
            --opt 'qualifiers: (l)-[:P580]->(st_time)'
            --where 'substr(kgtk_date_date(st_time), 2) < "2019-12-31" AND w.lang = "en"'
            --return 'n1 as node1, 
                      kgtk_lqstring_text(n1_label) as node1_label, 
                      "position held" as label, 
                      n2 as node2, 
                      kgtk_lqstring_text(n2_label) as node2_label, 
                      substr(kgtk_date_date(st_time), 2) as point_in_time'
            --order-by 'point_in_time desc'
            -o '../data/member_of_sports_teams_wikidata.tsv.gz'
""")

[2022-08-01 11:13:46 query]: SQL Translation:
---------------------------------------------
  SELECT graph_5_c1."node1" "_aLias.node1", kgtk_lqstring_text(graph_1_c2."node2") "_aLias.node1_label", ? "_aLias.label", graph_5_c1."node2" "_aLias.node2", kgtk_lqstring_text(graph_1_c3."node2") "_aLias.node2_label", substr(kgtk_date_date(graph_7_c5."node2"), ?) "_aLias.point_in_time"
     FROM graph_1 AS graph_1_c2
     INNER JOIN graph_1 AS graph_1_c3, graph_11 AS graph_11_c4, graph_5 AS graph_5_c1
     ON graph_5_c1."node1" = graph_11_c4."node1"
        AND graph_5_c1."node1" = graph_1_c2."node1"
        AND graph_5_c1."node2" = graph_1_c3."node1"
        AND graph_11_c4."label" = ?
        AND graph_5_c1."label" = ?
     LEFT JOIN graph_7 AS graph_7_c5
     ON graph_5_c1."id" = graph_7_c5."node1"
        AND graph_7_c5."label" = ?
        AND ((substr(kgtk_date_date(graph_7_c5."node2"), ?) < ?) AND (graph_11_c4."lang" = ?))
     ORDER BY "_aLias.point_in_time" DESC
  PARAS: ['position held

In [5]:
members_df = pd.read_csv('../data/member_of_sports_teams_wikidata.tsv.gz', sep='\t')

/Users/amandeep/anaconda3/envs/kgtk-env/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3361: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
print(len(members_df), len(members_df[members_df['point_in_time'] != '']), len(members_df[members_df['point_in_time'] == '']))

9705005 9705005 0


## Significant Event P793

In [7]:
kypher(""" --debug
            -i claims -i qualifiers -i label -i sitelinks --gc $STORE
            --match 'claims: (n1)-[l:P793]->(n2),
                     label: (n1)-[]->(n1_label),
                     label: (n2)-[]->(n2_label),
                     sitelinks: (n1)-[w:wikipedia_sitelink]->(wikipedia_url)'
            --opt 'qualifiers: (l)-[:P585]->(st_time)'
            --where 'substr(kgtk_date_date(st_time), 2) < "2019-12-31" AND w.lang = "en"'
            --return 'n1 as node1, 
                      kgtk_lqstring_text(n1_label) as node1_label, 
                      "position held" as label, 
                      n2 as node2, 
                      kgtk_lqstring_text(n2_label) as node2_label, 
                      substr(kgtk_date_date(st_time), 2) as point_in_time'
            --order-by 'point_in_time desc'
            -o '../data/significant_event_wikidata.tsv.gz'
""")

[2022-08-01 11:29:56 query]: SQL Translation:
---------------------------------------------
  SELECT graph_5_c1."node1" "_aLias.node1", kgtk_lqstring_text(graph_1_c2."node2") "_aLias.node1_label", ? "_aLias.label", graph_5_c1."node2" "_aLias.node2", kgtk_lqstring_text(graph_1_c3."node2") "_aLias.node2_label", substr(kgtk_date_date(graph_7_c5."node2"), ?) "_aLias.point_in_time"
     FROM graph_1 AS graph_1_c2
     INNER JOIN graph_1 AS graph_1_c3, graph_11 AS graph_11_c4, graph_5 AS graph_5_c1
     ON graph_5_c1."node1" = graph_11_c4."node1"
        AND graph_5_c1."node1" = graph_1_c2."node1"
        AND graph_5_c1."node2" = graph_1_c3."node1"
        AND graph_11_c4."label" = ?
        AND graph_5_c1."label" = ?
     LEFT JOIN graph_7 AS graph_7_c5
     ON graph_5_c1."id" = graph_7_c5."node1"
        AND graph_7_c5."label" = ?
        AND ((substr(kgtk_date_date(graph_7_c5."node2"), ?) < ?) AND (graph_11_c4."lang" = ?))
     ORDER BY "_aLias.point_in_time" DESC
  PARAS: ['position held

In [9]:
sig_df = pd.read_csv('../data/significant_event_wikidata.tsv.gz', sep='\t')

In [10]:
print(len(sig_df), len(sig_df[sig_df['point_in_time'] != '']), len(sig_df[sig_df['point_in_time'] == '']))

1240488 1240488 0


## Employer P108

In [12]:
kypher(""" --debug
            -i claims -i qualifiers -i label -i sitelinks --gc $STORE
            --match 'claims: (n1)-[l:P108]->(n2),
                     label: (n1)-[]->(n1_label),
                     label: (n2)-[]->(n2_label),
                     sitelinks: (n1)-[w:wikipedia_sitelink]->(wikipedia_url)'
            --opt 'qualifiers: (l)-[:P580]->(st_time)'
            --where 'substr(kgtk_date_date(st_time), 2) < "2019-12-31" AND w.lang = "en"'
            --return 'n1 as node1, 
                      kgtk_lqstring_text(n1_label) as node1_label, 
                      "position held" as label, 
                      n2 as node2, 
                      kgtk_lqstring_text(n2_label) as node2_label, 
                      substr(kgtk_date_date(st_time), 2) as start_time'
            --order-by 'start_time desc'
            -o '../data/employer_wikidata.tsv.gz'
""")

[2022-08-01 11:34:26 query]: SQL Translation:
---------------------------------------------
  SELECT graph_5_c1."node1" "_aLias.node1", kgtk_lqstring_text(graph_1_c2."node2") "_aLias.node1_label", ? "_aLias.label", graph_5_c1."node2" "_aLias.node2", kgtk_lqstring_text(graph_1_c3."node2") "_aLias.node2_label", substr(kgtk_date_date(graph_7_c5."node2"), ?) "_aLias.start_time"
     FROM graph_1 AS graph_1_c2
     INNER JOIN graph_1 AS graph_1_c3, graph_11 AS graph_11_c4, graph_5 AS graph_5_c1
     ON graph_5_c1."node1" = graph_11_c4."node1"
        AND graph_5_c1."node1" = graph_1_c2."node1"
        AND graph_5_c1."node2" = graph_1_c3."node1"
        AND graph_11_c4."label" = ?
        AND graph_5_c1."label" = ?
     LEFT JOIN graph_7 AS graph_7_c5
     ON graph_5_c1."id" = graph_7_c5."node1"
        AND graph_7_c5."label" = ?
        AND ((substr(kgtk_date_date(graph_7_c5."node2"), ?) < ?) AND (graph_11_c4."lang" = ?))
     ORDER BY "_aLias.start_time" DESC
  PARAS: ['position held', 2, 

In [13]:
employer_df = pd.read_csv('../data/employer_wikidata.tsv.gz', sep='\t')

In [15]:
print(len(employer_df), len(employer_df[employer_df['start_time'] != '']), len(employer_df[employer_df['start_time'] == '']))

1975754 1975754 0
